## Building a Chatbot

In this, we'll go over an example of how to design and implent an LLM -  powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that I build will only use the language model to have a converstion. There are several other related concepts that we may be looking for.

- Conversational RAG: Enable a chatbit exprience over an external source of data.
- Agents: Build a chatbot that can take actions.

This will helps adavanced topics.

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()  # loading all the env variables

groq_api_key = os.getenv("GROQ_API_KEY")


In [5]:
from langchain_groq import ChatGroq
model = ChatGroq(model="llama-3.1-8b-instant", groq_api_key = groq_api_key)
model

c:\Users\amank\.conda\envs\langchain_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x00000215CFF49090>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000215CFF48FA0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [6]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, My name is Aman and I am a AI Engineer.")])

AIMessage(content='Nice to meet you, Aman. As an AI Engineer, you must be working on some exciting projects involving artificial intelligence and machine learning. What specific areas of AI Engineering interest you the most, such as natural language processing, computer vision, or perhaps robotics?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 49, 'total_tokens': 102, 'completion_time': 0.083639759, 'completion_tokens_details': None, 'prompt_time': 0.003651814, 'prompt_tokens_details': None, 'queue_time': 0.056298186, 'total_time': 0.087291573}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f757f4b0bf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bb846-1b33-72f1-a98a-01c0415c46c3-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 49, 'output_tokens': 53, 'total_tokens': 102})

In [7]:
## Checking - Remembring the previous context..
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi, My name is Aman and I am a AI Engineer."),
        AIMessage(content='Nice to meet you, Aman. As an AI Engineer, you must be working on developing and implementing artificial intelligence models, integrating them with other systems, and ensuring their accuracy and efficiency.'),
        HumanMessage(content="Hey, what is my name and What do I do?")
    ]
)

AIMessage(content='Your name is Aman, and you are an AI Engineer.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 109, 'total_tokens': 123, 'completion_time': 0.016901918, 'completion_tokens_details': None, 'prompt_time': 0.006369919, 'prompt_tokens_details': None, 'queue_time': 0.050308105, 'total_time': 0.023271837}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bb846-1d06-7a91-9b0c-642390e97062-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 109, 'output_tokens': 14, 'total_tokens': 123})

### Message History

We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future intracttions will then load those messages and pass them into the chain as part of the input.

In [8]:
!pip install langchain_community

In [9]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

## store the chat-session
store = {}

## Separate the different sessions - This will distinguish one chat session with other
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id] 


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [10]:
## test with dummy config
config = {'configurable': {"session_id": "chat1"}}

In [14]:
## COmbining this chat session with this session Id -  Chat1
response = with_message_history.invoke(
    [HumanMessage(content="Hi, My name is Aman and I am  a AI Engineer.")],
    config=config
)

In [15]:
response.content

"Hello Aman, it's nice to meet you. As an AI Engineer, I'm curious - what sparked your interest in AI and machine learning? Was it a personal project, or perhaps a particular application of AI that caught your attention?"

In [ ]:
## session remembering the data with session id
with_message_history.invoke(
    [HumanMessage(content="What is my name ?")],
    config=config
)


AIMessage(content="Your name is Aman. You mentioned it earlier: you're a AI Engineer named Aman.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 195, 'total_tokens': 216, 'completion_time': 0.040653455, 'completion_tokens_details': None, 'prompt_time': 0.013866376, 'prompt_tokens_details': None, 'queue_time': 0.049866103, 'total_time': 0.054519831}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bb84a-9f9f-7261-89e0-fc6078fb7e96-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 195, 'output_tokens': 21, 'total_tokens': 216})

In [23]:
## Change the config --- changing the sessionId
config1 = {'configurable': {"session_id": "chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content = "What is my name ?")],
    config=config1
)

response.content

"I'm not aware of any information about your name. I'm a large language model, I don't have the ability to know your personal information unless you choose to share it with me. Each time you interact with me, it's a new conversation and I don't retain any information from previous conversations. If you'd like to share your name with me, I'd be happy to chat with you!"

In [26]:
## Now adding info with sessionId -  chat2
response = with_message_history.invoke(
    [HumanMessage(content = "My name is Steve.")],
    config=config1
)

In [27]:
config1 = {'configurable': {"session_id": "chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content = "What is my name ?")],
    config=config1
)

response.content

"Steve, it's still your name. I didn't forget."